In [ ]:
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/custom-EM-BERT/prof_entity/lightning

/content/gdrive/MyDrive/custom-EM-BERT/prof_entity/lightning


In [ ]:
!nvidia-smi

Fri May 28 02:03:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup

In [1]:
pip install jsonlines datasets pytorch_lightning transformers lightning_transformers ipython-autotime

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import jsonlines
from typing import Any, Dict, List, Optional
from datasets import Dataset, load_dataset, DatasetDict
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import BackboneFinetuning, BaseFinetuning
from torch.optim.optimizer import Optimizer
from pytorch_lightning.callbacks import ModelPruning, EarlyStopping, ModelCheckpoint
from lightning_transformers.core.nlp import HFBackboneConfig, HFTransformerDataConfig, HFDataModule
from lightning_transformers.task.nlp.text_classification import (
    TextClassificationDataModule, TextClassificationTransformer)
from transformers import AutoTokenizer, PreTrainedTokenizerBase, AutoModelForSequenceClassification
from sklearn.metrics import classification_report

# Lightning Modules

In [ ]:
# edited data module to only take in test data for prediction, as train set

class EntityMatchingDataModule(TextClassificationDataModule):
    def __init__(self,
                 cfg: HFTransformerDataConfig,
                 tokenizer: PreTrainedTokenizerBase,
                 train_data: pd.DataFrame):
        super().__init__(tokenizer, cfg)
        self.train_data = train_data

    def load_dataset(self) -> DatasetDict:
        return DatasetDict({
            'train': Dataset.from_pandas(train_data)})

    def process_data(self, dataset, stage: Optional[str] = None) -> Dataset:
        dataset = EntityMatchingDataModule.preprocess(
            dataset,
            tokenizer=self.tokenizer,
            padding=self.cfg.padding,
            truncation=self.cfg.truncation,
            max_length=self.cfg.max_length,
        )
        cols_to_keep = [
            x for x in ["input_ids", "attention_mask", "token_type_ids", "labels"] if x in dataset["train"].features
        ]
        dataset.set_format("torch", columns=cols_to_keep)
        self.labels = dataset["train"].features["labels"]
        self.labels.num_classes = len(dataset['train']['labels'].unique())
        return dataset

    @staticmethod
    def convert_to_features(
        example_batch: Any, _, tokenizer: PreTrainedTokenizerBase, **tokenizer_kwargs
    ):
    # for our use case, we will have to tokenize our 2 examples for entity matching
        return tokenizer(example_batch['descA'],
                         example_batch['descB'],
                         padding=True,
                         truncation=True)

    @staticmethod
    def preprocess(ds: Dataset, **fn_kwargs) -> Dataset:
        ds = ds.map(
            # todo: change this to self.convert_to_features for users to override
            EntityMatchingDataModule.convert_to_features,
            batched=True,
            with_indices=True,
            fn_kwargs=fn_kwargs,
        )
        ds.rename_column_("label", "labels")
        return ds

In [ ]:
class EntityMatcher(TextClassificationTransformer):
    def __init__(self, learning_rate=1e-5, max_lr=1e-3,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)

        # this is to initialize the backbone in this instance
        for k,v in kwargs.items():
            if k == 'backbone': 
                self.backbone = v

        self.lr = learning_rate
        self.max_lr = max_lr
        # self.num_classes = num_classes
        # self.save_hyperparameters()
    
    # this helps to hook num_classes to underlying model
    # without it, cannot call predict
    def setup(self, stage):
      self.model.num_classes=2
    
    def forward(self, x): # for inference
        # import pdb; pdb.set_trace()
        input_ids = x['input_ids']
        token_type_ids = x['token_type_ids']
        attention_mask = x['attention_mask']
        return self.model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )

    def predict_step(self, batch, batch_idx: int, dataloader_idx: int = None):
        return self(batch)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-5)

# Predict

In [ ]:
%load_ext autotime
# display time for each cell execution

time: 77.3 µs (started: 2021-05-28 02:03:45 +00:00)


In [ ]:
# load in test data under train
train_data = pd.read_csv('data/test.csv')

train_data.shape

(764, 4)

time: 297 ms (started: 2021-05-28 02:03:45 +00:00)


In [ ]:
model = EntityMatcher.load_from_checkpoint(checkpoint_path = 'bert_final.ckpt')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

time: 44.3 s (started: 2021-05-28 02:03:45 +00:00)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'bert-base-uncased')

data_loader = EntityMatchingDataModule(
    cfg=HFTransformerDataConfig(
        # num_workers=12,
        batch_size=8, # keep to max of 8, only use 16 with colab pro
        max_length=512),
    tokenizer=tokenizer,
    train_data=train_data)


time: 2.96 s (started: 2021-05-28 02:04:29 +00:00)


In [ ]:
trainer = pl.Trainer(
    gpus=1,
    precision=16, # change from fp32 to 16 for faster run-time
    progress_bar_refresh_rate=20, # slow down refresh rate for colab
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


time: 83.9 ms (started: 2021-05-28 02:04:32 +00:00)


In [ ]:
# call setup to initiate data loader without training step -- for inference
data_loader.setup()

# data is loaded under train, although is test set
test_loader = data_loader.train_dataloader()


time: 298 ms (started: 2021-05-28 02:04:33 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.


In [ ]:
predicted_values = trainer.predict(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



time: 22.2 s (started: 2021-05-28 02:04:33 +00:00)


In [ ]:
predictions = torch.cat([p.logits for p in predicted_values], dim=0)
preds = predictions.softmax(dim=1).argmax(dim=1).cpu()

time: 13.1 ms (started: 2021-05-28 02:04:55 +00:00)


In [ ]:
train_data['preds'] = preds
train_data.head()

,Unnamed: 0,descA,descB,label,preds
0,133,These clinical symptoms commonly occur in mult...,“BlackThorn is focused on developing a new gen...,1,0
1,2774,"The innovation labs we have funded, have creat...",Approximately 60% of these jobs are held by Si...,0,0
2,3102,"Mobile by design, Tabit is bringing the smartp...",At the 400 restaurants and cafes in Israel and...,0,0
3,537,LeaseLock secured $10M in Series A financing f...,"MARINA DEL REY, Calif., Aug 22, 2019 /PRNewswi...",1,1
4,1338,"Same-day services, including order pickups and...",And offline store sales actually grew 10% at a...,0,0


time: 22.3 ms (started: 2021-05-28 02:05:42 +00:00)


In [ ]:
train_data.to_csv('predictions.csv')